In [1]:
import docker

In [45]:
client = docker.DockerClient()

In [79]:
testdb = client.containers.run(
    'postgres:14',
    detach=True,
    environment={
        'ENVIRONMENT': 'development',
        'POSTGRES_USER': 'postgres',
        'POSTGRES_PASSWORD': 'postgres',
    },
    name='testdb',
    ports={5432: 5434},
    mem_limit='2g',
    auto_remove=True,
)


In [101]:
testdb.stop()

In [96]:
import psycopg2
connection = psycopg2.connect(user='postgres', password='postgres', host='localhost', port='5434', database='postgres')
cursor = conn.cursor()
create_table_query = '''CREATE TABLE mobile
          (ID INT PRIMARY KEY     NOT NULL,
          MODEL           TEXT    NOT NULL,
          PRICE         REAL); '''
    # Execute a command: this creates a new table
cursor.execute(create_table_query)
connection.commit()
# Executing a SQL query to insert data into  table
insert_query = """ INSERT INTO mobile (ID, MODEL, PRICE) VALUES (1, 'Iphone12', 1100)"""
cursor.execute(insert_query)
connection.commit()
print("1 Record inserted successfully")
# Fetch result
cursor.execute("SELECT * from mobile")
record = cursor.fetchall()
print("Result ", record)

# Executing a SQL query to update table
update_query = """Update mobile set price = 1500 where id = 1"""
cursor.execute(update_query)
connection.commit()
count = cursor.rowcount
print(count, "Record updated successfully ")
# Fetch result
cursor.execute("SELECT * from mobile")
print("Result ", cursor.fetchall())

# Executing a SQL query to delete table
delete_query = """Delete from mobile where id = 1"""
cursor.execute(delete_query)
connection.commit()
count = cursor.rowcount
print(count, "Record deleted successfully ")
# Fetch result
cursor.execute("SELECT * from mobile")
print("Result ", cursor.fetchall())


1 Record inserted successfully
Result  [(1, 'Iphone12', 1100.0)]
1 Record updated successfully 
Result  [(1, 'Iphone12', 1500.0)]
1 Record deleted successfully 
Result  []
